In [95]:
#https://www.datacamp.com/community/tutorials/fuzzy-string-python
#https://towardsdatascience.com/how-to-do-fuzzy-matching-in-python-pandas-dataframe-6ce3025834a6
#https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

In [103]:
OSM = pd.read_json('amenities-vancouver.json.gz', lines=True)
rest_list = ['cafe', 'fast_food', 'bbq', 'restaurant', 'pub',
       'bar', 'ice_cream', 'bistro', 'juice_bar']
restaurants = OSM[OSM.amenity.isin(rest_list)]
restaurants['name'] = restaurants['name'].str.lower()
restaurants = restaurants[~restaurants.name.isnull()].reset_index(drop=True) #remove null names
restaurants

<ipython-input-103-9b4fbb944ac1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants['name'] = restaurants['name'].str.lower()


,lat,lon,timestamp,amenity,name,tags
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ..."
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,salad loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...
2,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,best bite indian cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6..."
3,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,the cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li..."
4,49.265951,-123.246630,2011-11-19T08:06:36.000-08:00,pub,mahony and sons,{'wheelchair': 'yes'}
...,...,...,...,...,...,...
5057,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,house of dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-..."
5058,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,creekside coffee,{}
5059,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,togo sushi,{'cuisine': 'japanese'}
5060,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,brown's social house,"{'addr:housenumber': '215', 'brewery': 'Guinne..."


In [97]:
chains_text = open("canadian_chains.txt", "r")
chainList = chains_text.read().split('\n')
chainList = [item.lower() for item in chainList]
#chainList

In [104]:
matchList = []
ratioList = []
def match_chains(str2Match):
    ratio = process.extractOne(str2Match, chainList, scorer=fuzz.token_set_ratio, score_cutoff=90)
    if ratio == None:
        matchList.append(None)
        ratioList.append(None)
    else:
        matchList.append(ratio[0])
        ratioList.append(ratio[1])
    
ratios = np.vectorize(match_chains,otypes=[tuple])(restaurants['name'])

restaurants['match'] = matchList
restaurants['ratio'] = ratioList
restaurants


,lat,lon,timestamp,amenity,name,tags,match,ratio
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ...",starbucks,100.0
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,salad loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...,None,NaN
2,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,best bite indian cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6...",None,NaN
3,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,the cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li...",None,NaN
4,49.265951,-123.246630,2011-11-19T08:06:36.000-08:00,pub,mahony and sons,{'wheelchair': 'yes'},None,NaN
...,...,...,...,...,...,...,...,...
5057,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,house of dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-...",None,NaN
5058,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,creekside coffee,{},None,NaN
5059,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,togo sushi,{'cuisine': 'japanese'},togo sushi,100.0
5060,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,brown's social house,"{'addr:housenumber': '215', 'brewery': 'Guinne...",brown's social house,100.0


In [109]:
restaurants[(restaurants['ratio'] < 94)]

,lat,lon,timestamp,amenity,name,tags,match,ratio
59,49.286601,-123.128209,2019-08-23T07:46:30.000-07:00,cafe,chatime,"{'brand:wikidata': 'Q16829306', 'brand:en': 'C...",chattime,93.0
521,49.263828,-123.141379,2017-05-28T00:16:35.000-07:00,restaurant,earls,"{'opening_hours': 'Su-Th 11:30-24:00, Fr,Sa 11...",earl's,91.0
887,49.224633,-122.988360,2019-09-13T13:57:03.000-07:00,cafe,chatime,"{'brand:wikidata': 'Q16829306', 'brand:en': 'C...",chattime,93.0
1686,49.154142,-123.124046,2019-09-13T13:57:03.000-07:00,cafe,chatime,"{'addr:housenumber': '450-9100', 'addr:street'...",chattime,93.0
1691,49.227430,-123.116829,2019-09-13T13:57:03.000-07:00,cafe,chatime,"{'brand:wikidata': 'Q16829306', 'brand:en': 'C...",chattime,93.0
2073,49.282332,-123.134562,2013-05-26T17:40:58.000-07:00,bar,score,"{'addr:housenumber': '1262', 'website': 'http:...",scores,91.0
2590,49.266369,-123.246810,2018-08-23T04:02:34.000-07:00,cafe,j j bean,{},jj bean,92.0
2602,49.189186,-122.846313,2019-09-13T13:57:04.000-07:00,cafe,chatime,"{'brand:wikidata': 'Q16829306', 'brand:en': 'C...",chattime,93.0
2955,49.177490,-123.129286,2019-07-16T20:42:42.000-07:00,restaurant,phở hòa,"{'brand:wikidata': 'Q55629932', 'name:vi': 'Ph...",pháÿ hãa,91.0
2969,49.281461,-123.105171,2019-09-13T13:56:49.000-07:00,pub,darby's,"{'addr:housenumber': '18', 'website': 'https:/...",arby's,92.0
